In [2]:
import cv2
import os
import matplotlib.pyplot as plt 
import numpy as np


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Visualize the video using OpenCV
This code cell is useful for visualizing frames of a video, and the green rectangle can be adjusted to focus on a specific region of interest.

In [3]:

# Replace 'your_video_file.mp4' with the actual path to your video file
video_file_path = r'../videos_plateau_train_val/train.avi'

# Open the video file
cap = cv2.VideoCapture(video_file_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get the original frame dimensions
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Set the desired window dimensions
window_width = 800
window_height = int(window_width * (frame_height / frame_width))

# Create a window with the desired dimensions, only necessary as my display has low resolution 
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Video', window_width, window_height)


# Define the coordinates of the rectangle
rect_start = (int(0.35 * window_width), int(0.5 * window_height))
rect_end = (int(0.55 * window_width), int(0.75 * window_height))

# Read and display the video frame by frame
while True:
    # Read a frame
    ret, frame = cap.read()

    # If the frame is not read successfully, break the loop
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (window_width, window_height))
    # Draw the rectangle on the frame
    cv2.rectangle(frame, rect_start, rect_end, (0, 255, 0), 2)  # Green rectangle


    # Display the frame
    cv2.imshow('Video', frame)

    # Break the loop if the user presses 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

# Capture the image and put in a training dataset

In [4]:

# Replace 'your_video_file.mp4' with the actual path to your video file
video_file_path = r'../videos_plateau_train_val/train.avi'

# Define the coordinates of the rectangle
rect_start = (int(0.35 * 2448), int(0.5 * 2048))
rect_end = (int(0.55 * 2448), int(0.75 * 2048))

# Open the video file
cap = cv2.VideoCapture(video_file_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize frame counter
frame_count = 0

# Read and crop each frame, and save it to the output folder
while True:
    # Read a frame
    ret, frame = cap.read()

    # If the frame is not read successfully, break the loop
    if not ret:
        break

    # Crop the frame within the rectangle
    cropped_frame = frame[rect_start[1]:rect_end[1], rect_start[0]:rect_end[0]]
    cropped_frame = cv2.resize(cropped_frame, (224, 224))
    

    # Save the cropped frame to the output folder
    output_path = os.path.join('../All_image_data', f'frame_{frame_count}.png')
    cv2.imwrite(output_path, cropped_frame)

    # Increment frame counter
    frame_count += 1

    # Break the loop if the user presses 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()
